In [2]:
#Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(color_codes = True)

In [3]:
#Load Dataset
data = pd.read_csv('./mtcars.csv')
data.head()

model   mpg  cyl   disp   hp  drat     wt   qsec  vs  am  gear  \
0          Mazda RX4  21.0    6  160.0  110  3.90  2.620  16.46   0   1     4   
1      Mazda RX4 Wag  21.0    6  160.0  110  3.90  2.875  17.02   0   1     4   
2         Datsun 710  22.8    4  108.0   93  3.85  2.320  18.61   1   1     4   
3     Hornet 4 Drive  21.4    6  258.0  110  3.08  3.215  19.44   1   0     3   
4  Hornet Sportabout  18.7    8  360.0  175  3.15  3.440  17.02   0   0     3   

   carb  
0     4  
1     4  
2     1  
3     1  
4     2

In [4]:
#Select only mpg, cyl and hp values
data2 = data[['mpg','cyl','hp']]
data2.head()

mpg  cyl   hp
0  21.0    6  110
1  21.0    6  110
2  22.8    4   93
3  21.4    6  110
4  18.7    8  175

In [5]:
#Overview of Dataset Characteristics
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
mpg    32 non-null float64
cyl    32 non-null int64
hp     32 non-null int64
dtypes: float64(1), int64(2)
memory usage: 848.0 bytes


In [6]:
#Summary of N/A Values
data2.isnull().sum()

mpg    0
cyl    0
hp     0
dtype: int64

In [7]:
#Visualization of Correlations
sns.heatmap(data2.corr(), annot=True, cmap="Blues")

In [8]:
#Identify Correlated Columns
def correlation(dataset, threshold):
    col_corr = set() #set will contain unique values
    corr_matrix = dataset.corr() #finding correlation between columns
    for i in range (len(corr_matrix.columns)): #number of columns
        for j in range (i):
            if abs(corr_matrix.iloc[i,j])> threshold: #checking correlation between columns
                colName = corr_matrix.columns[i] #getting column name
                col_corr.add(colName) #adding the correlated columns name
    return col_corr #returning set of column names

col=correlation(data2.drop('mpg',axis = 1),0.8)
print('Correlated Columns:', col)

Correlated Columns: {'hp'}


BUILDING GENERAL MULTIVARIATE MODEL

In [11]:
#Define X and y variables
X = data2.drop('mpg', axis = 1).values
y = data2['mpg'].values

In [12]:
#Load Library for Training Dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 100)

In [13]:
#Script for Linear Regression Model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score

for name, method in [('Linear Regression', LinearRegression())]:
    method.fit(X_train, y_train)
    predict = method.predict(X_test)
print('Method: %s' %name)

Method: Linear Regression


In [14]:
#Coefficients 
print('Intercept: %.2f'% method.intercept_)
coeff_table=pd.DataFrame(np.transpose(method.coef_), data2.drop('mpg', axis = 1).columns, columns =['Coefficients'])
print(coeff_table)

Intercept: 36.41
     Coefficients
cyl     -2.100589
hp      -0.023089


In [15]:
#MAE, MSE and RMSE
print('R2: %.2f'% r2_score(y_test, predict))
print('Mean Absolute Error: %.2f'% metrics.mean_absolute_error(y_test, predict))
print('Mean Squared Error: %.2f'% metrics.mean_squared_error(y_test, predict))
print('Root Mean Squared Error: %.2f'% np.sqrt(metrics.mean_squared_error(y_test, predict)))

R2: 0.84
Mean Absolute Error: 1.41
Mean Squared Error: 4.15
Root Mean Squared Error: 2.04


OPTIMIZATION USING GRIDSEARCHCV

In [17]:
#Grid Search - Linear Model
from sklearn.model_selection import GridSearchCV

lm2 = LinearRegression()
param_grid = {'fit_intercept':[True,False],'normalize':[True,False], 'copy_X':[True,False]}
lm2_model = GridSearchCV(lm2, param_grid, cv=10, verbose = 0, scoring = 'neg_mean_squared_error', n_jobs=-1)

In [18]:
#Model Creation
from sklearn.metrics import accuracy_score, precision_score, recall_score
print('Optimized Models')

Optimized Models


In [23]:
#Fit Grid Search
lm2_model.fit(X_train, y_train)

#Best Params
print('Best Params: %s:' % lm2_model.best_params_)

#Predict on Test Data with Best Params
y_pred = lm2_model.predict(X_test)

#Test Data Accuracy of Model with Best Params
print('R2: %.2f'% r2_score(y_test, y_pred))
print('Intercept: %.2f'% lm2_model.best_estimator_.intercept_)

coeff_table = pd.DataFrame(np.transpose(lm2_model.best_estimator_.coef_),
                           data2.drop('mpg',axis=1).columns, columns=['Coefficients'])    
print(coeff_table)

Best Params: {'copy_X': True, 'fit_intercept': True, 'normalize': True}:
R2: 0.84
Intercept: 36.41
     Coefficients
cyl     -2.100589
hp      -0.023089


In [24]:
#MAE, MSE and RMSE
print('R2: %.2f'% r2_score(y_test, y_pred))
print('Mean Absolute Error: %.2f'% metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error: %.2f'% metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error: %.2f'% np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2: 0.84
Mean Absolute Error: 1.41
Mean Squared Error: 4.15
Root Mean Squared Error: 2.04
